In [2]:
# Full setup
import duckdb
import pandas as pd
from sodapy import Socrata
import config
import database_setup as dbs
from mta_dataset import *

%reload_ext sql
conn = dbs.get_database_connection(config.DEV_DATABASE)
%sql conn --alias duckdb


Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


## Misc

In [4]:
nearest_ada_stations_df = pd.read_csv('nearest_ada_stations.csv')
dbs.create_schema(conn, 'manual_entry')

table_name = 'manual_entry.nearest_ada_stations'
dbs.create_table(conn, nearest_ada_stations_df, table_name)
conn.close()

## Testing new filter on the Socrata Client

In [20]:
start_time = "'2023-01-02T00:00:00'"
end_time = "'2023-01-04T0:00:00'"

client = Socrata("data.ny.gov",
                     config.NY_OPEN_DATA_API_TOKEN,
                     username=config.NY_OPEN_DATA_USERNAME,
                     password=config.NY_OPEN_DATA_PASSWORD)

results = client.get(OriginDestination.code,
                    where=f"day_of_week in ('Monday', 'Tuesday') AND month = 1 AND year = 2023",
                    limit=100000000)


records_df = pd.DataFrame.from_records(results)

In [21]:
records_df.shape[0]

2881915

In [18]:
records_df.to_csv('test_size')